In [ ]:
!pip install transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

This approach is heavily based on https://link.springer.com/article/10.1007/s11257-022-09345-8#Tab3

Mainly used libraries: Pytorch, Transformers ||
Models: "CogComp/ZeroShotWiki" ||
Scrapped_Website_Url: https://smilesandgrins.com/


In [ ]:
!pip install torch
!pip install -U textblob

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
%cd '/content/drive/MyDrive/Recommend'

/content/drive/MyDrive/Recommend


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, nltk
from textblob import TextBlob

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("CogComp/ZeroShotWiki")
model = AutoModelForSequenceClassification.from_pretrained('CogComp/ZeroShotWiki')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def preprocess_data(data):
    # Clean the data (remove special characters, lowercase, etc.)
    data = [line.strip() for line in data]
    data = [line.lower() for line in data if line != '']
    return data

In [ ]:
with open("input.txt", "r") as f:
  data = f.readlines()

labels = ["orthodontist", "implant", "beauty-oriented", "narcissistic",
          "premium", "technically"]

classify_vectors = []
weight_vector = []

with torch.no_grad():
    for text in data:
        label_score = {}
        for label in labels:
            inputs = tokenizer(text, label, return_tensors='pt')
            out = model(**inputs)
            label_score[label]=float(torch.nn.functional.softmax(out[0],
                                                                 dim=-1)[0][0])
            print(label_score)
        blob = TextBlob(text)
        # Predict the label with the highest score
        classify_vector = [label_score[key] for key in label_score]
        classify_vectors.append(classify_vector)
        # Score the weight values by calculating the polarity of the sentences and setimental analysis
        weight_vector.append([blob.sentiment.polarity])


{'orthodontist': 0.20858311653137207}
{'orthodontist': 0.20858311653137207, 'implant': 0.35194528102874756}
{'orthodontist': 0.20858311653137207, 'implant': 0.35194528102874756, 'beauty-oriented': 0.21886372566223145}
{'orthodontist': 0.20858311653137207, 'implant': 0.35194528102874756, 'beauty-oriented': 0.21886372566223145, 'narcissistic': 0.1724303513765335}
{'orthodontist': 0.20858311653137207, 'implant': 0.35194528102874756, 'beauty-oriented': 0.21886372566223145, 'narcissistic': 0.1724303513765335, 'premium': 0.0773133635520935}
{'orthodontist': 0.20858311653137207, 'implant': 0.35194528102874756, 'beauty-oriented': 0.21886372566223145, 'narcissistic': 0.1724303513765335, 'premium': 0.0773133635520935, 'technically': 0.22219300270080566}
{'orthodontist': 0.9938730001449585}
{'orthodontist': 0.9938730001449585, 'implant': 0.8872222900390625}
{'orthodontist': 0.9938730001449585, 'implant': 0.8872222900390625, 'beauty-oriented': 0.4758020043373108}
{'orthodontist': 0.993873000144958

In [ ]:
classify_tensor = torch.tensor(classify_vectors)
transposed_classified_tensor = torch.transpose(classify_tensor, 0, 1)
weight_tensor = torch.tensor(weight_vector)
# Get the result tensor by mutipling the score_tensor and weight_tensor
result_tensor = torch.mm(transposed_classified_tensor, weight_tensor)
print(result_tensor)

tensor([[13.8650],
        [16.0655],
        [ 8.4106],
        [ 3.0776],
        [ 8.4872],
        [ 8.1026]])


In [ ]:
# Normalizing the vectors

min_val = result_tensor.min()
max_val = result_tensor.max()

# Normalize the tensor to the range [0, 1]
normalized_tensor = (result_tensor - min_val) / (max_val - min_val)

# Scale the normalized tensor to the range [1, 10]
scaled_tensor = 1 + normalized_tensor * 9

print(scaled_tensor)

tensor([[ 8.4751],
        [10.0000],
        [ 4.6955],
        [ 1.0000],
        [ 4.7486],
        [ 4.4821]])


In [ ]:
for i in range(len(labels)):
  print(labels[i], int(scaled_tensor[i]))

orthodontist 8
implant 10
beauty-oriented 4
narcissistic 1
premium 4
technically 4
